<style type="text/css" media="screen">
a:link { color:#FFFFFF; text-decoration: none; }
a:visited { color:#FFFFFF; text-decoration: none; }
a:hover { color:#FFFFFF; text-decoration: none; }
a:active { color:#FFFFFF; text-decoration: underline; }
</style>
# <a id="sqld" style="text-decoration:none;">HPE CSI Driver Data Fabric for Kubernetes</a>
![](imgs/hpedevlogo.svg)
<img src="imgs/mapr-ar21.gif" alt="csi logo" style="float: right;">
<img src="imgs/csilogo1.png" alt="csi logo" style="float: right;">
## How to deploy SQL Server 2019 with an Operator on Kubernetes Cluster

Emmanuel COLUSSI 
<br>
<br>
<br>
<br>

## Kubernetes Deployments
<p>The Deployment instructs Kubernetes how to create and update instances of your application. Once you've created a Deployment, the Kubernetes master schedules mentioned application instances onto individual Nodes in the cluster.</p>
<p>Once the application instances are created, a Kubernetes Deployment Controller continuously monitors those instances. If the Node hosting an instance goes down or is deleted, the Deployment controller replaces the instance with an instance on another Node in the cluster. This provides a self-healing mechanism to address machine failure or maintenance.</p>
<img src="imgs/kubdeploy.jpg" alt="kubdeploy" style="display: block;margin-left: auto;margin-right: auto;width:40%;">

## Deploying SQL Server workload
### Benefits of running SQL Server in containers
The first and most basic benefit of using containers is the simplicity and elegance of the solution. There is no need to install SQL Server or configure a failover cluster. You can deploy SQL Server containers with a single command, and Kubernetes inherently provides high availability for your SQL Server deployments.</br>
In some cases, the availability of SQL Server instances in a container deployed on Kubernetes could be even higher than that of a workload deployed on top of a failover cluster.</br>
However, the main benefit of running SQL Server in containers lies in high-density deployments and resource sharing. A fundamental difference between containers and virtual machines (VMs) is that unlike VMs, containers are not restricted to a fixed amount of resources for the duration that they are running. A shared pool of resources is often thrown at a group of containers running side by side on the same host. </br>
This enables each container to consume more or fewer resources at different points in time. As long as the aggregate consumption is less than the amount of resources available in the pool, all containers get all the resources they need.</br></br>

![](imgs/sqla.gif)



<p>By containerizing MSSQL with MapR , customers have all the benefits of MSSQL, MapR, and Docker combined.</br>
Here, MSSQL offers robust RDBM services that persist data into MapR for disaster recovery and data protection while leveraging kubernetes technologies for scalability and agility. </p>

![](imgs/sqlmapr1.png)

<link rel="stylesheet" type="text/css" href="css/bootstrap.min.css">
<link rel="stylesheet" type="text/css" href="css/fontawesome.min.css">

In this example, the container hosting the SQL Server instance is described as a Kubernetes deployment object. The deployment creates a replica set. The replica set creates the pod.</br>
In this step, create a deployment file to describe the container based on the SQL Server mssql-server-linux Docker image. The deployment file references the mssql-server persistent volume claim, and the mssql secret that you already applied to the Kubernetes cluster. The manifest also describes a service. This service is a load balancer. The load balancer guarantees that the IP address persists after SQL Server instance is recovered.</br>
</br>
<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>
> In the first step we will create a secret file for the deployment of the instance,that configures the container image to set the SA password.</br><b>password </b>string (SHBlaW52ZW50) is a base64 encoded.
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-cogs"></i>&nbsp;&nbsp;secret file example : create-secret-mssql-student4.yaml</p>
 
</div>


<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> </p>
  
  
</div>

In [ ]:
ssh -t  $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/create-mssql-secret-$JUPYTERHUB_USER.yaml -n mapr-$JUPYTERHUB_USER"

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-user-secret"></i>&nbsp;&nbsp;You should now see the sqlsecret-secret</p>
    
 
</div>

In [ ]:
ssh -t  $JUPYTERHUB_USER@arvine "kubectl  get secret/sqlsecret-$JUPYTERHUB_USER -n mapr-$JUPYTERHUB_USER"

<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>
> SQL Server 2019 containers makes it easier and simpler to work with data, and monthly preview releases offer the latest innovation and improvements.SQL Server 2019 containers safer by starting the SQL Server process as a non-root user by default.</br>

> The application process within most Docker containers is running as a root user meaning the process has root privileges within the container user space. The root user within the container is also the same root (uid 0) on the host machine, and if the user can break out of the container, they would have root permissions on the host. Running as root is convenient for development, testing and CI/CD use cases but for production use cases, it is safest to run SQL Server as a non-root process within the container </br>

> You must therefore create a "user and group" <b>mssql</b> with home directory <b>/var/opt/mssql</b> on each node of the cluster.
In our configuration the user and the group <b>mssql</b> are already created on each node


##### In our example we will create a deployment file for SQL Server 2019 with the main following parameters : 
<table class="table" style="font-size: 14px;">
  <tbody>
    <tr>
        <td style="width:40%"><b>Deployment / name :</b> </td> <td>Deployment  Name</td>
    </tr>
      <tr>
        <td style="width:40%"><b>secretKeyRef / runAsUser :</b> </td> <td>ID for mssql user</td>
    </tr>
      <tr>
        <td style="width:40%"><b>secretKeyRef / fsGroup :</b> </td> <td>ID for mssql group</td>
    </tr>
      <tr>
        <td style="width:40%"><b>image :</b></td><td>SQL Server Docker image to use</td>
    </tr>
    <tr>
        <td style="width:40%"><b>claimName :</b></td><td>Name of PVC to use</td>
    </tr>
      <tr>
        <td style="width:40%"><b>MSSQL_PID :</b></td><td>Developer: Sets the container to run SQL Server Developer edition. Developer edition is not licensed for production data. If the deployment is for production use, set the appropriate edition (Enterprise, Standard, or Express).</td>
    </tr>
    <tr>
        <td style="width:40%"><b>Service / name :</b></td><td>Service Name : mssql-service-student4</td>
    </tr>
 </tbody>
</table>  
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-file-export"></i>&nbsp;&nbsp;Deployement file example : deploy-mssql-s.yaml</p>
 
</div>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> </p>
   <hr>
  <p class="mb-0">Or connect to the hosts 'bandol' using the terminal at the bottom of the window</p>
 
</div>

In [ ]:
ssh -t  $JUPYTERHUB_USER@arvine "kubectl create -f sql-mapr/deploy-mssql-s-$JUPYTERHUB_USER.yaml -n mapr-$JUPYTERHUB_USER"

The deployment and service are created. The SQL Server instance is in a container, connected to persistent storage.
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-check-circle"></i>&nbsp;&nbsp;To view the status of the pod</p>
    <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i> </p>
 
</div>

In [ ]:
ssh -t  $JUPYTERHUB_USER@arvine "kubectl  get pods -n mapr-$JUPYTERHUB_USER"

In the preceding result Cell , the pod has a status of Running. This status indicates that the container is ready. This may take several minutes.</br>
<i class="fas fa-2x fa-info-circle" style="color:#FFAD33;"></i>
> After the deployment is created, it can take a few minutes before the pod is visible. The delay is because the cluster pulls the mssql-server-linux image from the Docker hub. After the image is pulled the first time, subsequent deployments might be faster if the deployment is to a node that already has the image cached on it.</br></br>

<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
    <p><i class="fas fa-check-circle"></i>&nbsp;&nbsp;Verify the service is running : the name of SQL Service is <b><i>mssql-service-studentxx</i></b></p>
    <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the command in the next Cell with button <i class="fas fa-play"></i></p>
 
</div>

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "kubectl  get services -n mapr-$JUPYTERHUB_USER"

### Connect to the SQL Server 

Now you can connect to the SQL Server instance outside the Kubernetes cluster with the following tools:
<ul class="list-group">
  <li class="list-group-item"><i class="fas fa-tasks" style="color:#FFAD33;"></i>&nbsp;&nbsp;SSMS</li>
  <li class="list-group-item"><i class="fas fa-database" style="color:#FFAD33;"></i>&nbsp;&nbsp;Azure Data Studio</li>
  <li class="list-group-item"><i class="fas fa-terminal" style="color:#FFAD33;"></i>&nbsp;&nbsp;sqlcmd</li>
  <li class="list-group-item"> <i class="fas fa-tools" style="color:#FFAD33;"></i>&nbsp;&nbsp;Other tools ...</li>
</ul>
<i class="fas fa-2x fa-hand-point-right" style="color:#FFAD33;"></i>&nbsp;&nbsp;For the connection you use the ip address of one of the nodes of the Kubernetes cluster with TCP port number.</br>
In this example the port is <b>30418</b>




<ul class="list-group">
    <li class="list-group-item"><i class="fas fa-server" style="color:#FFAD33;"></i>&nbsp;&nbsp;arvine 
    <li class="list-group-item"><i class="fas fa-server" style="color:#FFAD33;"></i>&nbsp;&nbsp;garanoir
    <li class="list-group-item"><i class="fas fa-server" style="color:#FFAD33;"></i>&nbsp;&nbsp;gamaret
    <li class="list-group-item"><i class="fas fa-server" style="color:#FFAD33;"></i>&nbsp;&nbsp;syrah   
</ul>
<div class="alert" style="background-color:#FFAD33;color:#fff;" role="alert">
  
  <p><i class="fas fa-plug"></i>&nbsp;&nbsp;Connect to the SQL Server instance with sqlcmd tool. But you can use another tool like SSMS, Azure Data Studio ....</p>
  <p class="mb-0">set <b><i>HOST_NAME</i></b> with your hosts IP Address value and TCP Port in next Cell with button <i class="fas fa-play"></i> : export HOST_NAME=Your_value</p>  
   <hr>
    <p class="mb-0"><i class="fas fa-sync-alt"></i>&nbsp;&nbsp;Run the commands in the next Cell with button <i class="fas fa-play"></i> </p>
 
</div>

In [ ]:
export HOST_NAME="arvine,30351"

In [ ]:
ssh -t $JUPYTERHUB_USER@arvine "/opt/mssql-tools/bin/sqlcmd -U sa -P HPeinvent@ -S $HOST_NAME -i sql-mapr/getversion.sql"

</br></br>
<div class="alert" style="background-color:#FFAD33;color:#fff;width:100%" role="alert">
  <p style="font-size:20px;display: block;text-align:center;margin-left: auto;margin-right: auto;"><i class="fas fa-4x fa-smile"></i>&nbsp;&nbsp;Congratulations on completing the module&nbsp;&nbsp;<i class="fas fa-4x fa-praying-hands"></i></p>
</div>  

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#FFAD33;"></i>&nbsp;&nbsp;Next Steps

Restore SQL Server Database


 <a href="HPEMAPR-SQL-DB.ipynb#restore" target="New" title="Next: Restore Database"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
 <a href="HPEMAPR-SQL-PVC.ipynb#pvc" target="New" title="Back: Create Persitent Volume"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#FFAD33;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>

<p style="display: block;margin-left: auto;margin-right: auto;width:10%;" ><a href="#sqld" title="TOP"><i class="fas fa-2x fa-chevron-circle-up" style="color:#FFAD33;"></i></a></p>